In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [2]:
!pip install tensorflow==1.13.1

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/train_khan_acad.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/test_khan_acad.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/val_khan_acad.csv" /content


In [5]:
!cp -r "/content/drive/My Drive/Information_retrieval_project/khan_acad/IR_project_khan_acad_model_hyperbolic_hinge_5" /content/

In [ ]:
!pip install transformers==3.2.0

In [7]:
!pip install git+https://github.com/geoopt/geoopt.git
! pip install git+https://github.com/ferrine/hyrnn.git

  Cloning https://github.com/geoopt/geoopt.git to /tmp/pip-req-build-rt5ccgnq
  Running command git clone -q https://github.com/geoopt/geoopt.git /tmp/pip-req-build-rt5ccgnq
  Created wheel for geoopt: filename=geoopt-0.3.1-cp37-none-any.whl size=76194 sha256=fd06d50220a169af9a4d90ede43aec67f46028811f93dfaa38571c0f9d018cfa
  Stored in directory: /tmp/pip-ephem-wheel-cache-dtniyks_/wheels/10/df/30/e0d857f034c142ca5f38af048b62aae3da773b272553e5dd21
Successfully built geoopt
  Cloning https://github.com/ferrine/hyrnn.git to /tmp/pip-req-build-boebzfe4
  Running command git clone -q https://github.com/ferrine/hyrnn.git /tmp/pip-req-build-boebzfe4
  Created wheel for hyrnn: filename=hyrnn-0.0.0-cp37-none-any.whl size=13955 sha256=a2403d5d3990010fbcfd7603e3b9ed14708ff316af02ae2789eb4d1a30f8252c
  Stored in directory: /tmp/pip-ephem-wheel-cache-nu_jnpx_/wheels/24/c3/64/cc0e9d25d466081dc154a2a8843157f54d845b916b4ba66418
Successfully built hyrnn


In [8]:
import pandas as pd
train_data = pd.read_csv("train_khan_acad.csv")
val_data = pd.read_csv("val_khan_acad.csv")
test_data = pd.read_csv("test_khan_acad.csv")

train_data


,video_transcripts,hierarchy
0,In the last couple of videos we saw that we c...,math>>multivariable-calculus>>multivariable-de...
1,- What we're going to do in this video is gi...,science>>ap-biology>>natural-selection
2,"So once again, we have three equal, or we say...",math>>pre-algebra>>pre-algebra-equations-expre...
3,- Liz's math test included a survey question...,math>>engageny-alg-1>>alg1-2
4,- The following two equations form a linear s...,math>>algebra-home>>alg-system-of-equations
...,...,...
4183,- Hello everyone. So this is what I might ca...,math>>multivariable-calculus>>multivariable-de...
4184,- Let's try now to subtract some two-digit n...,math>>early-math>>cc-early-math-add-sub-100
4185,- Let's say that I have a circle. My best att...,math>>engageny-geo>>geo-5
4186,- So let's look at the female reproductive cy...,science>>health-and-medicine>>human-anatomy-an...


In [9]:
set(train_data["video_transcripts"].values).intersection(set(test_data["video_transcripts"].values))

set()

In [10]:
from google.colab import files

In [11]:
import re
def clean_sentence(question):
  # print(question)
  question = re.sub('<[^>]*>', ' ',question)
  question = re.sub(' +', ' ', question)
  question = re.sub('\xa0','',question)
  question = question.rstrip()
  question = re.sub('nan','',question)
  question = re.sub(u'\u2004','',question)
  question = re.sub(u'\u2009','',question)

  # question = question.decode("utf-8")
  # question = question.replace(u'\u200\d*','').encode("utf-8")
  question = re.sub('&nbsp','',question)
  question = re.sub('&ndash','',question)
  question = re.sub('\r','',question)
  question = re.sub('\t','',question)
  question = re.sub('\n',' ',question)

  question = re.sub('MathType@.*','',question)
  question = re.sub('&thinsp','',question)
  question = re.sub('&times','',question)
  question = re.sub('\u200b','',question)
  question = re.sub('&rarr;;;','',question)

  return question

In [12]:
# final_data_1 = final_data.loc[0:71003,:]
# final_data_1

In [13]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Loading BERT tokenizer...


In [14]:
train_data["video_transcripts"].value_counts()

 [This video contains no audio narrations.]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [15]:

# from sklearn.preprocessing import LabelEncoder

# LE = LabelEncoder()
# final_data['label'] = LE.fit_transform(final_data['board_syllabus'])
# final_data.head()

In [16]:
# def get_labels(prediction):
#     predicted_label =  LE.inverse_transform([prediction])
#     return predicted_label[0]

In [17]:
# get_labels(330)

In [18]:




# train_data = pd.concat([train_data,val_data])
# train_data

In [19]:
train_features = train_data["video_transcripts"]
test_features = test_data["video_transcripts"]
train_labels = train_data["hierarchy"]
test_labels = test_data["hierarchy"]
val_features = val_data["video_transcripts"]
val_labels = val_data["hierarchy"]

In [20]:
train_labels.value_counts()

science>>health-and-medicine>>circulatory-system-diseases                           99
science>>health-and-medicine>>human-anatomy-and-physiology                          65
science>>health-and-medicine>>respiratory-system-diseases                           55
science>>health-and-medicine>>circulatory-system                                    54
science>>health-and-medicine>>infectious-diseases                                   52
                                                                                    ..
math>>algebra-basics>>alg-basics-expressions-with-exponents                          1
math>>old-integral-calculus>>riemann-sums-ic                                         1
math>>cc-third-grade-math>>imp-addition-and-subtraction                              1
math>>7th-grade-illustrative-math>>unit-2-introducing-proportional-relationships     1
science>>chemistry>>states-of-matter-and-intermolecular-forces                       1
Name: hierarchy, Length: 569, dtype: int64

In [21]:
test_labels.value_counts()

science>>health-and-medicine>>human-anatomy-and-physiology                24
science>>health-and-medicine>>circulatory-system-diseases                 22
science>>health-and-medicine>>circulatory-system                          17
science>>biology>>crash-course-bio-ecology                                11
science>>electrical-engineering>>robots                                   11
                                                                          ..
math>>early-math>>cc-early-math-add-sub-basics                             1
math>>in-in-class-7-math-india-icse>>in-in-7-exponents-and-powers-icse     1
science>>physics>>light-waves                                              1
science>>health-and-medicine>>health-care-system                           1
math>>algebra-home>>alg-conic-sections                                     1
Name: hierarchy, Length: 416, dtype: int64

In [22]:
question_answer = train_features.values
categories = train_labels.values

In [23]:
question_answer

array([" In the last couple of videos we saw that we can describe a curves by a position vector-valued function. And in very general terms, it would be the x position as a function of time times the unit vector in the horizontal direction. Plus the y position as a function of time times the unit victor in the vertical direction. And this will essentially describe this-- though, if you can imagine a particle and let's say the parameter t represents time. It'll describe where the particle is at any given time. And if we wanted a particular curve we can say, well, this only applies for some curve-- we're dealing, it's r of t. And it's only applicable between t being greater than a and less than b. And you know, that would describe some curve in two dimensions. Just me just draw it here. This is all a review of really, the last two videos. So this curve, it might look something like that where this is where t is equal to a. That's where t is equal to b. And so r of a will be this vector ri

In [24]:
categories

array(['math>>multivariable-calculus>>multivariable-derivatives',
       'science>>ap-biology>>natural-selection',
       'math>>pre-algebra>>pre-algebra-equations-expressions', ...,
       'math>>engageny-geo>>geo-5',
       'science>>health-and-medicine>>human-anatomy-and-physiology',
       'math>>multivariable-calculus>>greens-theorem-and-stokes-theorem'],
      dtype=object)

In [25]:
# val_features = test_features.values
# val_labels = test_labels.values

In [26]:
# this method can be used to project from euclidean space to hyperbolic space
def exponential_map(vector):
        norm_v = np.linalg.norm(vector, axis=1)
        coef = np.tanh(norm_v) / norm_v
        second_term = vector * coef[:, None]
        return second_term

In [27]:
# this method can be used to project from euclidean space to hyperbolic space
def tensor_exponential_map(vector):
      vector_norm = vector.norm(dim=-1, p=2, keepdim=True).clamp_min(1e-15)
      gamma_1 = torch.nn.functional.tanh(vector_norm) * (vector / vector_norm)
      return gamma_1

In [28]:

import numpy as np

from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
from bokeh.models import HoverTool
from bokeh import colors

from gzip import open as gopen

import gensim.models.poincare as poincare
poincare_model = poincare.PoincareModel.load("taxonomy_khan_acad_embedding_20.pkl")
def get_poincare_embeddings_data(taxonomy):
  cleaned_taxonomy = []
  for value in taxonomy:
      value = value.split(">>")
      cleaned_taxonomy.append( list(tok.lower() for tok in value) )
  return cleaned_taxonomy

In [29]:

# course_taxonomy

poincare_emb_data = get_poincare_embeddings_data(categories)
poincare_val = get_poincare_embeddings_data(val_labels)

In [30]:
# poincare_embedding =  [[ poincare_model.kv.get_vector(str(x)) for x in taxonomy ] for taxonomy in poincare_emb_data ]
# np.linalg.norm(poincare_embedding[1000])

In [31]:
# for index,embedding in enumerate(poincare_embedding):
#   poincare_embedding[index] = np.stack(embedding,axis=0)


In [32]:
# np.array(poincare_embedding).shape

In [33]:
poincare_embedding =  [exponential_map(np.expand_dims( np.hstack(  [ poincare_model.kv.get_vector(str(x)) for x in taxonomy ] ),axis=0)) for taxonomy in poincare_emb_data ]
np.linalg.norm(poincare_embedding[1000])

0.899765215781272

In [34]:
poincare_embedding_val = [exponential_map(np.expand_dims( np.hstack(  [ poincare_model.kv.get_vector(str(x)) for x in taxonomy ] ),axis=0)) for taxonomy in poincare_val ]
np.linalg.norm(poincare_embedding_val[100])

0.8857759138716806

In [35]:
poincare_val[100]

['math', 'cc-seventh-grade-math', 'cc-7th-ratio-proportion']

In [36]:
poincare_embedding_val[100].shape

(1, 60)

In [37]:
poincare_embedding[0].shape

(1, 60)

In [38]:
max_val_train = 0
max_emb =None
for embedding in poincare_embedding:
  val = embedding.shape[1]
  if val >max_val_train:
    max_val_train=val
    max_emb =embedding
max_val_train


60

In [39]:
max_val_val = 0
max_emb =None
for embedding in poincare_embedding_val:
  val = embedding.shape[1]
  if val >max_val_val:
    max_val_val=val
    max_emb =embedding
max_val_val


60

In [40]:
def get_concat_embedding(poincare_embedding,max_val):
  concatenated_embedding = []
  for embedding in poincare_embedding:
    if embedding.shape[1] < max_val:
      new_embedding = np.append(embedding, np.expand_dims(np.zeros(max_val-embedding.shape[1]),axis=0),axis=1)
    else:
      new_embedding = embedding
    concatenated_embedding.append(np.squeeze(new_embedding,axis=0))
  return concatenated_embedding



In [41]:
concat_embedding_train = get_concat_embedding(poincare_embedding,max_val_train)
concat_embedding_val = get_concat_embedding(poincare_embedding_val,max_val_train)

In [42]:
concat_embedding_train[0].shape

(60,)

In [43]:
poincare_embeddings_train = np.stack(concat_embedding_train, axis=0)
poincare_embeddings_train.shape

(4188, 60)

In [44]:
poincare_embeddings_val = np.stack(concat_embedding_val, axis=0).squeeze()
poincare_embeddings_val.shape

(924, 60)

In [45]:
input_ids = []
attention_masks = []

for sent in question_answer:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', question_answer[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:   In the last couple of videos we saw that we can describe a curves by a position vector-valued function. And in very general terms, it would be the x position as a function of time times the unit vector in the horizontal direction. Plus the y position as a function of time times the unit victor in the vertical direction. And this will essentially describe this-- though, if you can imagine a particle and let's say the parameter t represents time. It'll describe where the particle is at any given time. And if we wanted a particular curve we can say, well, this only applies for some curve-- we're dealing, it's r of t. And it's only applicable between t being greater than a and less than b. And you know, that would describe some curve in two dimensions. Just me just draw it here. This is all a review of really, the last two videos. So this curve, it might look something like that where this is where t is equal to a. That's where t is equal to b. And so r of a will be this vector

In [46]:
num_classes = len(list(set(categories)))
num_classes

569

In [47]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split

batch_size = 32


In [48]:
import numpy as np
from torch.autograd import Function
class Distance(Function):
    @staticmethod
    def grad(x, v, sqnormx, sqnormv, sqdist, eps):
        alpha = (1 - sqnormx)
        beta = (1 - sqnormv)
        z = 1 + 2 * sqdist / (alpha * beta)
        a = ((sqnormv - 2 * torch.sum(x * v, dim=-1) + 1) / torch.pow(alpha, 2))\
            .unsqueeze(-1).expand_as(x)
        a = a * x - v / alpha.unsqueeze(-1).expand_as(v)
        z = torch.sqrt(torch.pow(z, 2) - 1)
        z = torch.clamp(z * beta, min=eps).unsqueeze(-1)
        return 4 * a / z.expand_as(x)

    @staticmethod
    def forward(ctx, u, v, eps):
        squnorm = torch.clamp(torch.sum(u * u, dim=-1), 0, 1 - eps)
        sqvnorm = torch.clamp(torch.sum(v * v, dim=-1), 0, 1 - eps)
        sqdist = torch.sum(torch.pow(u - v, 2), dim=-1)
        ctx.eps = eps
        ctx.save_for_backward(u, v, squnorm, sqvnorm, sqdist)
        x = sqdist / ((1 - squnorm) * (1 - sqvnorm)) * 2 + 1
        # arcosh
        z = torch.sqrt(torch.pow(x, 2) - 1)
        return torch.log(x + z)

    @staticmethod
    def backward(ctx, g):
        u, v, squnorm, sqvnorm, sqdist = ctx.saved_tensors
        g = g.unsqueeze(-1)
        gu = Distance.grad(u, v, squnorm, sqvnorm, sqdist, ctx.eps)
        gv = Distance.grad(v, u, sqvnorm, squnorm, sqdist, ctx.eps)
        return g.expand_as(gu) * gu, g.expand_as(gv) * gv, None
def distanceTo(vector1,vector2):
        return Distance.apply(vector1,vector2,1e-5)
        # vector1 = vector1.detach().cpu().numpy()
        # vector2 = vector2.detach().cpu().numpy()
        # euclidean_dists = np.linalg.norm(vector1 - vector2)  
        # gamma = 1 + 2 * ((euclidean_dists ** 2) / ((1-(np.linalg.norm(vector1))) * (1-np.linalg.norm(vector2))))  # (1 + neg_size, batch_size)
        # poincare_dists = np.arccosh(gamma) 
        # return torch.tensor(poincare_dists,dtype=torch.float)

In [49]:

import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint
import geoopt.manifolds.stereographic.math as pm
import geoopt.optim.rsgd as rsgd_
import geoopt.optim.radam as radam_
# from hyrnn.nets import MobiusLinear
from geoopt.tensor import ManifoldParameter
from geoopt.manifolds.stereographic import PoincareBall
from tqdm import tqdm
import geoopt
import time
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint
import geoopt.optim.rsgd as rsgd_
import geoopt.optim.radam as radam_
# from hyrnn.nets import MobiusLinear
from geoopt.tensor import ManifoldParameter
import time
import argparse
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

def create_ball(ball=None, c=None):
    """
    Helper to create a PoincareBall.
    Sometimes you may want to share a manifold across layers, e.g. you are using scaled PoincareBall.
    In this case you will require same curvature parameters for different layers or end up with nans.
    Parameters
    ----------
    ball : geoopt.PoincareBall
    c : float
    Returns
    -------
    geoopt.PoincareBall
    """
    if ball is None:
        assert c is not None, "curvature of the ball should be explicitly specified"
        ball = geoopt.PoincareBall(c)
    # else trust input
    return ball


class MobiusLinear(torch.nn.Linear):
    def __init__(self, *args, nonlin=None, ball=None, c=1.0, **kwargs):
        super().__init__(*args, **kwargs)
        # for manifolds that have parameters like Poincare Ball
        # we have to attach them to the closure Module.
        # It is hard to implement device allocation for manifolds in other case.
        self.ball = create_ball(ball, c)
        if self.bias is not None:
            self.bias = geoopt.ManifoldParameter(self.bias, manifold=self.ball)
        self.nonlin = nonlin
        self.reset_parameters()

    def forward(self, input):
        return mobius_linear(
            input,
            weight=self.weight,
            bias=self.bias,
            nonlin=self.nonlin,
            ball=self.ball,
        )

    @torch.no_grad()
    def reset_parameters(self):
        torch.nn.init.eye_(self.weight)
        self.weight.add_(torch.rand_like(self.weight).mul_(1e-3))
        if self.bias is not None:
            self.bias.zero_()


# package.nn.functional.py
def mobius_linear(input, weight, bias=None, nonlin=None, *, ball: geoopt.PoincareBall):
    output = ball.mobius_matvec(weight, input)
    if bias is not None:
        output = ball.mobius_add(output, bias)
    if nonlin is not None:
        output = ball.logmap0(output)
        output = nonlin(output)
        output = ball.expmap0(output)
    return output
# Neural Classifierwork
class MulticlassClassifier(nn.Module):
    def __init__(self,bert_model_path):
        super(MulticlassClassifier,self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_path,output_hidden_states=False,output_attentions=False)
        for param in self.bert.parameters():
          param.requires_grad=True
        self.dropout = nn.Dropout(0.1)
        self.fc1 = MobiusLinear(768, 384)
        self.fc2 = MobiusLinear(384, 60)

    def forward(self,tokens,masks):
        _, pooled_output = self.bert(tokens, attention_mask=masks)
        hyerbolic_transform = tensor_exponential_map(pooled_output)
        x = self.fc1(hyerbolic_transform)
        x = self.fc2(x)
        return x

class MyHingeLoss(torch.nn.Module):
    def __init__(self, margin):
        super(MyHingeLoss, self).__init__()
        self.margin = margin


    def forward(self, output, target):
        loss = 0
        for i in range(len(output)):
            text_emb = output[i]
            t_label = target[i]
            j = randint(0, len(output)-1)
            while j == i:
                j = randint(0, len(output)-1)
            t_j = target[j]
            loss += torch.relu( self.margin + \
                            distanceTo(t_label, text_emb) - distanceTo(t_j, text_emb) )
        return loss / len(output)

class MyHingeLoss_cos(torch.nn.Module):

    def __init__(self, margin):
        super(MyHingeLoss_cos, self).__init__()
        self.margin = margin

    def forward(self, output, target):
        loss = 0
        for i in range(len(output)):
            text_emb = output[i]
            t_label = target[i]
            j = randint(0, len(output)-1)
            while j == i:
                j = randint(0, len(output)-1)
            t_j = target[j]
            loss += torch.relu( self.margin - cos(t_label, text_emb) + cos(t_j, text_emb) )
        return loss / len(output)

In [50]:
from transformers import BertModel, AdamW, BertConfig
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


In [51]:
from transformers import BertModel, AdamW, BertConfig

# Loads BertModel, the pretrained BERT model with a single 
model = MulticlassClassifier('bert-base-uncased')
model.load_state_dict(torch.load("IR_project_khan_acad_model_hyperbolic_hinge_5/model_weights"))
# Tell pytorch to run this model on the GPU.
model.cuda()


MulticlassClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [52]:
test_features = test_features.values
labels = test_labels.values

In [53]:
test_features

array([' -  What I hope to do in this video is get familiar with the notion of an interval, and also think about ways that we can show an interval, or interval notation. Right over here I have a number line. Let\'s say I wanted to talk about the interval on the number line that goes from negative three to two. So I care about this-- Let me use a different color. Let\'s say I care about this interval right over here. I care about all the numbers from negative three to two. So in order to be more precise, I have to be clear. Am I including negative three and two, or am I not including negative three and two, or maybe I\'m just including one of them. So if I\'m including negative three and two, then I would fill them in. So this right over here, I\'m filling negative three and two in, which means that negative three and two are part of this interval. And when you include the endpoints, this is called a closed interval. Closed interval. And I just showed you how I can depict it on a number

In [54]:
labels

array(['math>>math1>>x89d82521517266d4:functions',
       'math>>old-ap-calculus-ab>>ab-applications-definite-integrals',
       'economics-finance-domain>>ap-macroeconomics>>economic-iondicators-and-the-business-cycle',
       ...,
       'science>>in-in-class11th-physics>>in-in-system-of-particles-and-rotational-motion',
       'math>>precalculus>>x9e81a4f98389efdf:prob-comb',
       'science>>biology>>dna-as-the-genetic-material'], dtype=object)

In [55]:
len(list(set(labels)))

416

In [56]:

# course_taxonomy
test_labels = list(set(labels))
poincare_emb_data = get_poincare_embeddings_data(test_labels)

In [57]:
poincare_emb_data

[['math',
  'in-in-class-8-math-india-hindi',
  'in-in-8-algebraic-expressions-and-identities-hindi'],
 ['math', 'arithmetic-home', 'arith-place-value'],
 ['math', 'integral-calculus', 'ic-integration'],
 ['math', 'engageny-alg-1', 'alg1-2'],
 ['math', 'calculus-all-old', 'derivative-applications-calc'],
 ['math',
  'statistics-probability',
  'significance-tests-confidence-intervals-two-samples'],
 ['math', 'algebra-home', 'alg-functions'],
 ['economics-finance-domain', 'ap-microeconomics', 'imperfect-competition'],
 ['math',
  'calculus-2',
  'cs2-parametric-equations-polar-coordinates-and-vector-valued-functions'],
 ['math', 'algebra', 'x2f8bb11595b61c86:systems-of-equations'],
 ['math', 'ap-statistics', 'two-sample-inference'],
 ['math', 'old-ap-calculus-ab', 'ab-derivatives-advanced'],
 ['math', 'geometry', 'hs-geo-transformations'],
 ['math', 'in-in-grade-12-ncert', 'in-in-probability-of-events'],
 ['math', 'calculus-2', 'cs2-integration-techniques'],
 ['math', 'pre-algebra', 'pr

In [58]:
poincare_embedding =  [exponential_map(np.expand_dims( np.hstack(  [ poincare_model.kv.get_vector(str(x)) for x in taxonomy ] ),axis=0)) for taxonomy in poincare_emb_data ]
np.linalg.norm(poincare_embedding[311])

0.8812958075961795

In [59]:
max_val = 0
max_emb =None
for embedding in poincare_embedding:
  val = embedding.shape[1]
  if val >max_val:
    max_val=val
    max_emb =embedding
max_val


60

In [60]:
concatenated_embedding = []
for embedding in poincare_embedding:
  if embedding.shape[1] < max_val_train:
    new_embedding = np.append(embedding, np.expand_dims(np.zeros(max_val_train-embedding.shape[1]),axis=0),axis=1)
  else:
    new_embedding = embedding
  concatenated_embedding.append(np.squeeze(new_embedding,axis=0))



In [61]:
poincare_embeddings_final = np.stack(concatenated_embedding, axis=0)
poincare_embeddings_final.shape

(416, 60)

In [62]:
# model2 = MulticlassClassifier('bert-base-uncased')
# model2.load_state_dict(torch.load('model_hyperbolic_round_2/model_weights'))
# model2.cuda()

In [63]:
test_input_ids = []
test_attention_masks = []
for sent in test_features:

    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
# labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  
test_poincare_tensor = torch.tensor(poincare_embeddings_final,dtype=torch.float)

# Create the DataLoader.
# prediction_data = TensorDataset(test_input_ids, test_attention_masks, test_poincare_tensor)
# prediction_sampler = SequentialSampler(prediction_data)
# prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [64]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
test_poincare_tensor = torch.tensor(poincare_embeddings_final,dtype=torch.float)


In [65]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
def dist_without_grad( u, v):
  sqdist = torch.sum((u - v) ** 2, dim=-1)
  squnorm = torch.sum(u ** 2, dim=-1)
  sqvnorm = torch.sum(v ** 2, dim=-1)
  x = 1 + 2 * sqdist / ((1 - squnorm) * (1 - sqvnorm)) + 1e-7
  z = torch.sqrt(x ** 2 - 1)
  return torch.log(x + z)

In [66]:
test_labels = np.array(test_labels)

In [97]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(test_input_ids)))
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

# Put model in evaluation mode
model.eval()
input_ids = test_input_ids.to('cuda')
attention_masks = test_attention_masks.to('cuda')
test_poincare_tensor = test_poincare_tensor.to('cuda')
# Tracking variables 
predictions , true_labels = [], []
for input_id,attention_mask in zip(input_ids, attention_masks):
  with torch.no_grad():
    outputs = model(input_id.reshape(1,-1),attention_mask.reshape(1,-1))
  distances,indices = torch.topk(dist_without_grad(outputs,test_poincare_tensor),5,largest=False)
  predictions.append(test_labels[indices.cpu().numpy()])
print(len(predictions))
  # max_distance =100000000000000
  # label=None
  # for index,test_poincare in enumerate(test_poincare_tensor):

  #   distance = distanceTo(test_poincare, outputs)
  #   if distance < max_distance:
  #     max_distance = distance
  #     label = index
  # predictions.append(labels[label])
    
# Predict 
# for batch in prediction_dataloader:
#   # Add batch to GPU
#   batch = tuple(t.to(device) for t in batch)
  
#   # Unpack the inputs from our dataloader
#   b_input_ids, b_input_mask, b_labels = batch
  
#   # Telling the model not to compute or store gradients, saving memory and 
#   # speeding up prediction
#   with torch.no_grad():
#       # Forward pass, calculate logit predictions
#       outputs = model(b_input_ids,b_input_mask)

#   logits = outputs
#   for logit in logits:
#     max_similarity = 0


#   # Move logits and labels to CPU
#   logits = logits.detach().cpu().numpy()
#   label_ids = b_labels.to('cpu').numpy()
  
#   # Store predictions and true labels
#   predictions.append(logits)
#   true_labels.append(label_ids)

print('    DONE.')
# predictions

Predicting labels for 1,047 test sentences...


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


1047
    DONE.


In [98]:
!pip install tensorflow==1.13.1

In [99]:
labels=test_data['hierarchy'].values

In [100]:
for index,label in enumerate(labels):
  if label not in list(train_data["hierarchy"].values):
    print(index,label)


61 math>>ap-statistics>>density-curves-normal-distribution-ap
314 math>>ap-statistics>>density-curves-normal-distribution-ap
385 math>>arithmetic>>fraction-arithmetic
436 math>>old-integral-calculus>>convergence-tests-integral-calc
444 math>>ap-statistics>>density-curves-normal-distribution-ap
879 math>>old-integral-calculus>>convergence-tests-integral-calc
926 math>>arithmetic>>fraction-arithmetic


In [101]:
from sklearn .preprocessing import LabelEncoder
LE= LabelEncoder()
labels = LE.fit_transform(labels)
labels

array([238, 254,   2, ..., 386, 283, 324])

In [102]:
labels

array([238, 254,   2, ..., 386, 283, 324])

In [103]:
np.where(labels==325)

(array([ 787, 1034]),)

In [104]:
final_predictions = []
for prediction in predictions:
  final_predictions.append(LE.transform(prediction))


In [105]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 20
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, k=20)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, k=20)

tmp_rank = tf.nn.top_k(y_pred, 20)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))


    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 20) (1047,)
update_recall:  0.5616045845272206
recall 0.5616045845272206
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 588.0, 459.0, 0.0, 0.0]
TMP_RANK:  TopKV2(values=array([[297, 296, 295, ..., 190, 189,  29],
       [259, 258, 257, ...,  82,  81,  77],
       [ 20,  19,  18, ...,   2,   1,   0],
       ...,
       [182, 181, 180, ...,  92,  91,  47],
       [284, 283, 282, ...,  80,  79,  77],
       [334, 332, 331, ..., 301, 300, 299]]), indices=array([[14,  9,  5, ..., 18, 16, 19],
       [ 8, 12, 10, ..., 17, 14, 18],
       [15, 19, 18, ...,  8,  7, 10],
       ...,
       [ 0,  3,  4, ..., 13, 10, 19],
       [ 4,  3,  2, ...,  9, 13, 16],
       [12, 13, 14, ...,  3,  0,  2]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 15)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 15)

tmp_rank = tf.nn.top_k(y_pred, 15)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 15) (1047,)
update_recall:  0.5243553008595988
recall 0.5243553008595988
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 549.0, 498.0, 0.0, 0.0]
TMP_RANK:  TopKV2(values=array([[243, 242, 241, ..., 177, 176,  29],
       [226, 225, 224, ...,  89,  88,  87],
       [ 20,  19,  18, ...,   2,   1,   0],
       ...,
       [276, 182, 181, ...,  93,  92,  91],
       [284, 283, 282, ...,  79,  78,  77],
       [334, 332, 331, ..., 301, 300, 299]]), indices=array([[ 5,  6,  4, ...,  7,  9,  3],
       [ 2,  0,  1, ...,  9,  7, 10],
       [ 8, 13, 14, ...,  9,  7, 12],
       ...,
       [14,  2,  5, ...,  9, 13, 10],
       [ 4,  3,  2, ..., 10, 12, 13],
       [12, 13, 14, ...,  3,  0,  2]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 10)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 10)

tmp_rank = tf.nn.top_k(y_pred, 10)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 10) (1047,)
update_recall:  0.45367717287488063
recall 0.45367717287488063
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 475.0, 572.0, 0.0, 0.0]
TMP_RANK:  TopKV2(values=array([[296, 295, 292, ..., 243, 242, 241],
       [259, 254, 253, ...,  89,  88,  87],
       [ 16,  15,  14, ...,   3,   2,   1],
       ...,
       [182, 181, 180, ...,  95,  94,  93],
       [284, 283, 282, ...,  83,  81,  80],
       [327, 326, 323, ..., 301, 300, 299]]), indices=array([[9, 5, 4, ..., 1, 2, 0],
       [8, 9, 7, ..., 4, 6, 3],
       [5, 4, 3, ..., 9, 8, 7],
       ...,
       [0, 3, 4, ..., 8, 7, 9],
       [4, 3, 2, ..., 8, 7, 9],
       [9, 8, 7, ..., 3, 0, 2]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 5)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 5)

tmp_rank = tf.nn.top_k(y_pred, 5)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 5) (1047,)
update_recall:  0.3037249283667622
recall 0.3037249283667622
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 318.0, 729.0, 0.0, 0.0]
TMP_RANK:  TopKV2(values=array([[241, 191, 190, 189,  29],
       [226, 225, 224, 133, 132],
       [ 15,  14,  13,  11,  10],
       ...,
       [182, 180, 179, 178, 177],
       [284, 283, 282, 281, 280],
       [305, 302, 301, 300, 299]]), indices=array([[4, 1, 2, 0, 3],
       [2, 0, 1, 4, 3],
       [4, 3, 2, 0, 1],
       ...,
       [2, 4, 0, 1, 3],
       [4, 3, 2, 1, 0],
       [1, 4, 3, 0, 2]], dtype=int32))
